In [2]:
import pandas as pd
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
#import geopandas as gpd
import geopy as gp
from geopy.geocoders import GoogleV3

In [3]:
df = pd.read_csv(r'/Users/kamil/Documents/data_mining/reduced_data.csv')

In [4]:
df = df[df.Value > 0]

In [5]:
df["temp"] = np.nan

In [6]:
df_grouped = df.groupby(['Latitude', 'Longitude', 'Height']).agg({'temp':'size', 'Value':'mean'}).rename(columns={'Value':'Value_mean','temp':'number of measurements'}).reset_index()

In [7]:
df_grouped

,Latitude,Longitude,Height,number of measurements,Value_mean
0,-90.000000,37.442836,5.0,2,10.000000
1,-47.392879,127.472214,173.0,20,25.250000
2,-36.852563,174.763318,82.0,1,47.000000
3,-36.852563,174.763340,82.0,1,46.000000
4,-36.852542,174.763392,81.0,1,49.000000
...,...,...,...,...,...
70440,52.395588,4.875916,103.0,14,21.928571
70441,52.395607,4.874018,-32.0,257,25.599222
70442,60.202004,24.921344,44.0,1,54.000000
70443,90.000000,123.456000,123.0,1,123.000000


In [8]:
df_grouped.sort_values(by = ['Latitude', 'Longitude'], inplace = True)

In [9]:
df_grouped

,Latitude,Longitude,Height,number of measurements,Value_mean
0,-90.000000,37.442836,5.0,2,10.000000
1,-47.392879,127.472214,173.0,20,25.250000
2,-36.852563,174.763318,82.0,1,47.000000
3,-36.852563,174.763340,82.0,1,46.000000
4,-36.852542,174.763392,81.0,1,49.000000
...,...,...,...,...,...
70440,52.395588,4.875916,103.0,14,21.928571
70441,52.395607,4.874018,-32.0,257,25.599222
70442,60.202004,24.921344,44.0,1,54.000000
70443,90.000000,123.456000,123.0,1,123.000000


In [10]:
def get_country(row):
    locator = Nominatim(user_agent="ekspllo3", timeout=10)
    pos = str(row['Latitude']) + ', ' + str(row['Longitude'])
    location = locator.reverse(pos, timeout = 10, language="en")
    try:
      return location.raw['address']['country']
    except:
      return "no data" 

In [11]:
import math
import statistics
from tqdm import tqdm
data = []

Latitude, Longitude = 0, 0
countries = []
curr_country = "no data"

#df__dict = df.to_dict('records')
df_grouped_dict = df_grouped.to_dict('records')

for row in tqdm(df_grouped_dict):
    if Latitude == 0 and Longitude == 0:
      Latitude, Longitude = row['Latitude'], row['Longitude']
      curr_country = get_country(row)
      countries.append(curr_country)
    else:
        distance = math.sqrt((Latitude - row['Latitude'])**2 + (Longitude- row['Longitude'])**2) 
        if distance < 0.5:  #wtedy są blisko i można potraktować je z pewnością jako ten sam kraj (czyli przyłączamy wartości średnie do)
            countries.append(curr_country)
        else:

            Latitude, Longitude = row['Latitude'], row['Longitude']
            curr_country = get_country(row)
            countries.append(curr_country)

cdf = df_grouped
cdf['Country'] = countries


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70445/70445 [00:51<00:00, 1376.41it/s]


In [12]:
print(cdf)

        Latitude   Longitude  Height  number of measurements  Value_mean  \
0     -90.000000   37.442836     5.0                       2   10.000000   
1     -47.392879  127.472214   173.0                      20   25.250000   
2     -36.852563  174.763318    82.0                       1   47.000000   
3     -36.852563  174.763340    82.0                       1   46.000000   
4     -36.852542  174.763392    81.0                       1   49.000000   
...          ...         ...     ...                     ...         ...   
70440  52.395588    4.875916   103.0                      14   21.928571   
70441  52.395607    4.874018   -32.0                     257   25.599222   
70442  60.202004   24.921344    44.0                       1   54.000000   
70443  90.000000  123.456000   123.0                       1  123.000000   
70444  90.000000  140.471962    60.0                   21647   30.988451   

           Country  
0          no data  
1          no data  
2      New Zealand  
3  

In [13]:
cdf.sort_values(by = ['Country'], inplace = True)

In [14]:
cdf['Country'].value_counts()

United States    61223
Japan             8167
South Korea        452
New Zealand        247
China              159
Netherlands        121
Slovenia            54
no data              9
Canada               4
Germany              4
Italy                2
Finland              1
Somalia              1
Spain                1
Name: Country, dtype: int64

In [15]:
countries = cdf['Country'].unique()

In [16]:
countries

array(['Canada', 'China', 'Finland', 'Germany', 'Italy', 'Japan',
       'Netherlands', 'New Zealand', 'Slovenia', 'Somalia', 'South Korea',
       'Spain', 'United States', 'no data'], dtype=object)

In [17]:
cdf.describe()

,Latitude,Longitude,Height,number of measurements,Value_mean
count,70445.000000,70445.000000,70445.000000,7.044500e+04,70445.000000
mean,35.748421,-82.930392,104.024998,5.082449e+02,57.047007
std,5.202132,86.442267,117.307459,1.847460e+04,249.638167
min,-90.000000,-122.851453,-1809.000000,1.000000e+00,1.000000
25%,34.122169,-122.128059,20.000000,2.000000e+00,24.750000
50%,34.176193,-118.261475,104.000000,4.000000e+00,30.250000
75%,37.738979,-118.093719,162.000000,3.600000e+01,34.050000
max,90.000000,174.765607,6228.000000,1.257088e+06,33475.310345


In [18]:
def get_info(df):
  print(f"Total number of measurements : {df['number of measurements'].sum()}")
  print(f"Min Latitude : {df['Latitude'].min()} | Max Latitude : {df['Latitude'].max()}")
  print(f"Min Longitude : {df['Longitude'].min()} | Max Longitude : {df['Longitude'].max()}")
  print(f"Min Height : {df['Height'].min()} | Max Height : {df['Height'].max()}")
  print(f"Mean value of radiation : {df['Value_mean'].mean()}")
  print(f"Max value of radiation : {df['Value_mean'].max()}")

In [19]:
for country in countries:
  print(country + ":" + '\n')
  get_info(cdf[cdf.Country == country])
  print('\n' + "-" *50 + '\n')

Canada:

Total number of measurements : 4
Min Latitude : 43.3902971572416 | Max Latitude : 43.3920435590073
Min Longitude : -79.8252136094009 | Max Longitude : -79.8239368779098
Min Height : 2.0 | Max Height : 2.0
Mean value of radiation : 124.5
Max value of radiation : 337.0

--------------------------------------------------

China:

Total number of measurements : 31011
Min Latitude : 22.284233 | Max Latitude : 22.312658
Min Longitude : 114.145004 | Max Longitude : 114.184113
Min Height : 2.0 | Max Height : 199.0
Mean value of radiation : 71.66703198524368
Max value of radiation : 89.65

--------------------------------------------------

Finland:

Total number of measurements : 1
Min Latitude : 60.2020037715231 | Max Latitude : 60.2020037715231
Min Longitude : 24.9213437877441 | Max Longitude : 24.9213437877441
Min Height : 44.0 | Max Height : 44.0
Mean value of radiation : 54.0
Max value of radiation : 54.0

--------------------------------------------------

Germany:

Total number

In [23]:
cdf = cdf.reset_index(drop=True)

In [31]:
cdf

,Latitude,Longitude,Height,number of measurements,Value_mean,Country
0,43.392044,-79.824387,2.0,1,52.000000,Canada
1,43.391997,-79.825214,2.0,1,71.000000,Canada
2,43.391092,-79.823964,2.0,1,337.000000,Canada
3,43.390297,-79.823937,2.0,1,38.000000,Canada
4,22.285723,114.146713,151.0,8,70.000000,China
...,...,...,...,...,...,...
70440,0.000000,0.000000,1.0,1,1.000000,no data
70441,0.000000,-102.000000,96.0,8511,27.688168,no data
70442,-47.392879,127.472214,173.0,20,25.250000,no data
70443,90.000000,123.456000,123.0,1,123.000000,no data


In [95]:
formated_cdf_group = cdf.groupby(['Country'])
formated_cdf = formated_cdf_group.max()
formated_cdf = formated_cdf.reset_index()
formated_cdf['size'] = formated_cdf["Value_mean"].pow(0.3)
formated_cdf

,Country,Latitude,Longitude,Height,number of measurements,Value_mean,size
0,Canada,43.392044,-79.823937,2.0,1,337.000000,5.731773
1,China,22.312658,114.184113,199.0,2042,89.650000,3.852699
2,Finland,60.202004,24.921344,44.0,1,54.000000,3.309163
3,Germany,51.029021,13.706970,120.0,18706,31.167861,2.806158
4,Italy,45.773048,13.634098,15.0,126,26.698413,2.678833
5,Japan,37.930088,141.036057,1596.0,1257088,33475.310345,22.772850
6,Netherlands,52.395607,4.988073,103.0,20311,30.823529,2.796821
7,New Zealand,-36.851360,174.765607,279.0,9,49.000000,3.214096
8,Slovenia,46.565235,15.658566,323.0,13074,43.614350,3.103765
9,Somalia,1.809340,41.235100,66.0,255757,107.520490,4.068622


In [92]:
fig = px.scatter_geo(formated_cdf,
                    lat = formated_cdf['Latitude'],
                    lon = formated_cdf['Longitude'],
                    locationmode='country names',
                    color="Value_mean",
                    range_color= [0, max(formated_cdf['Value_mean'])+2],
                    hover_name="Country",
                    hover_data=["Value_mean"],
                    size="size",
                    projection="natural earth")
fig.show()

In [96]:
formated_cdf_group = cdf.groupby(['Country'])
formated_cdf = formated_cdf_group.mean()
formated_cdf = formated_cdf.reset_index()
formated_cdf['size'] = formated_cdf["Value_mean"].pow(0.3)
formated_cdf

,Country,Latitude,Longitude,Height,number of measurements,Value_mean,size
0,Canada,43.391357,-79.824375,2.000000,1.000000,124.500000,4.251584
1,China,22.285994,114.146685,148.911950,195.037736,71.667032,3.602432
2,Finland,60.202004,24.921344,44.000000,1.000000,54.000000,3.309163
3,Germany,50.219822,9.447700,101.000000,4678.750000,19.375298,2.433182
4,Italy,45.769642,13.616818,9.000000,64.000000,25.099206,2.629650
5,Japan,36.244732,139.797929,119.386678,2788.284805,271.284309,5.370657
6,Netherlands,52.276022,4.907893,7.743802,577.057851,25.806710,2.651672
7,New Zealand,-36.852149,174.763057,151.344130,1.072874,32.620670,2.844775
8,Slovenia,46.555011,15.636386,256.722222,472.203704,28.102194,2.720333
9,Somalia,1.809340,41.235100,66.000000,255757.000000,107.520490,4.068622


In [97]:
fig = px.scatter_geo(formated_cdf,
                    lat = formated_cdf['Latitude'],
                    lon = formated_cdf['Longitude'],
                    locationmode='country names',
                    color="Value_mean",
                    range_color= [0, max(formated_cdf['Value_mean'])+2],
                    hover_name="Country",
                    hover_data=["Value_mean"],
                    size="size",
                    projection="natural earth")
fig.show()

In [ ]:
formated_cdf_group = cdf.groupby(['Country'])
formated_cdf = formated_cdf_group.mean()
formated_cdf = formated_cdf.reset_index()
formated_cdf['size'] = formated_cdf["Value_mean"].pow(0.3)
formated_cdf